In [1]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as colors
import scipy.io as sio
import cmaps
from matplotlib.colors import LinearSegmentedColormap

In [3]:
import roms_workspace as rw
import scipy as sci
import os

In [11]:
my_rootnl = '/Volumes/Elements SE/roms4dvar_ecs'
nl_workspace_info = {
                     'freerun_detide_v3.7/outputs':'ROMS 3.7',
                     # 'freerun_detide_v4.1/outputs':'ROMS 4.1 NO SCORRECTION',
                        # 'freerun_detide_v4.1_SC/outputs':'ROMS 4.1 SCORRECTION',
                    # 'run17_v4/outputs':'FORWARD drggrd',
}
var = 'salt'
areas = {
        #  'bh': ('Bohai Sea', [0,7]),
        # 'ys': ('Yellow Sea', [6.5,11]),
        # 'cre': ('Changjiang River Estuary',[10.5,14.5]),
        # 'ks': ('Kuroshio Current System', [21.5,24]),
        'all': ('East China Sea', [10,25])
}
nl_Workspaces = []
for i in range(len(nl_workspace_info)):
    nl_Workspaces.append(rw.Nl_workspace( MY_ROOT         =my_rootnl,
                                    workspace_name  =list(nl_workspace_info.keys())[i],
                                    workspace_detail=list(nl_workspace_info.values())[i]
                                   ))
    nl_Workspaces[i].load_mtfroms(isAna=False,Surf=True)

Workspace: /Volumes/Elements SE/roms4dvar_ecs/freerun_detide_v3.7/outputs


In [12]:
var = 'salt'

In [14]:
nl_Vars = {}
for iw in range(len((nl_workspace_info))):
    for area in list(areas.keys()):
        print('loading %s in %s  area...'%(var,area))
        if iw == 0:
            nl_Vars[area] = []
        # loading data from FORWARD model outputs
        nl_Vars[area].append(nl_Workspaces[iw].pull_var2d(varname=         var, area=area,time_range=('2012-07-25','2012-08-01')))

loading salt in all  area...


In [28]:
len(nl_Workspaces)

2

In [9]:
def _forward(x):
    return x**3


def _inverse(x):
    return np.power(x,1/3)
norm = colors.FuncNorm((_forward, _inverse), vmin=5, vmax=35)

In [20]:
lon_bdy = [120,133.5];
lat_bdy = [20,31];
vmin_sst = 22
vmax_sst = 26
my_sst_color = sio.loadmat('LYG_rainbow.mat')['rainbow']
my_sst = LinearSegmentedColormap.from_list('sst',my_sst_color, N = 256)
my_div_color = np.array(  [
                 [0,0,123],
                [9,32,154],
                [22,58,179],
                [34,84,204],
                [47,109,230],
                [63,135,247],
                [95,160,248],
                [137,186,249],
                [182,213,251],
                [228,240,254],
                [255,255,255],
                [250,224,224],
                [242,164,162],
                [237,117,113],
                [235,76,67],
                [233,52,37],
                [212,45,31],
                [188,39,26],
                [164,33,21],
                [140,26,17],
                [117,20,12]
                ])/255
my_div = LinearSegmentedColormap.from_list('div',my_div_color, N = 256)


d_interval = 2

# iwda = 0
plt.rcParams['font.family'] = 'Lucida Grande'
area = 'all'
# for area in ['bh','ys','ks',]:
figsize = (11,5)
lon_bdy = [116,136];
lat_bdy = [21,38.5];
lon_interval = 5
lat_interval = 5
vmin_sst = 30
vmax_sst = 35
norm = colors.FuncNorm((_forward, _inverse), vmin=vmin_sst, vmax=vmax_sst)
vmin_diff = -0.5
vmax_diff = 0.5
d_interval = 5
bc = 0.93
for iwork in range(len(nl_Workspaces)):
    for it in range(len(nl_Vars[area][iwork].ocean_time)):
    # for it in range(1):
        print(iwork,it)
        fig = plt.figure(figsize=figsize, constrained_layout = True)
        #===============================observation spatial pattern
        ax = fig.add_subplot(projection = ccrs.Mercator())
        for bnd in ax.spines.keys():
            ax.spines[bnd].set_linewidth('1.5')
        ax.add_feature(cfeature.COASTLINE.with_scale('10m'), color = 'gray',zorder = 2,linewidth =1,)
        ax.add_feature(cfeature.LAND.with_scale('10m'),facecolor='lightgray',zorder = 1,)
        ax.set_extent((lon_bdy[0],lon_bdy[-1],lat_bdy[0],lat_bdy[-1]),crs = ccrs.PlateCarree())
        ax.set_xticks(np.arange(lon_bdy[0],lon_bdy[-1]+lon_interval,lon_interval),crs=ccrs.PlateCarree())
        ax.set_yticks(np.arange(lat_bdy[0],lat_bdy[-1]+lat_interval,lat_interval),crs=ccrs.PlateCarree())
        lon_formatter = LongitudeFormatter(number_format='.0f',
                                        degree_symbol='°',
                                        dateline_direction_label=True)
        lat_formatter = LatitudeFormatter(number_format='.0f',
                                            degree_symbol='°')
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)
        ax.tick_params(direction = 'out', length = 3, width = 1.5, labelsize = 12)
        c = ax.pcolormesh(nl_Vars[area][iwork].lon,nl_Vars[area][iwork].lat,
                  nl_Vars[area][iwork].value[it,:,:],
                          cmap = cmaps.NCV_bright,
                  # cmap = my_sst, 
                  # norm = norm,
                  # cmap = my_sst,
                          vmin = vmin_sst,vmax=vmax_sst,

               transform = ccrs.PlateCarree(),zorder = -1)
        ax.set_title('SSS '+nl_Workspaces[iwork].workspace_detail,fontsize=16)
        ax.text(lon_bdy[-1]*bc,lat_bdy[-1]*0.99,str(nl_Vars[area][iwork].ocean_time.data[it])[:10]+'-%sH'%(str(nl_Vars[area][iwork].ocean_time.data[it])[11:13]),fontsize = 12,transform = ccrs.PlateCarree())
        position=fig.add_axes([0.335,-0.05, 0.325, 0.045,])
        cb = fig.colorbar(c, cax = position, orientation='horizontal',ticks = np.arange(vmin_sst,vmax_sst+d_interval,d_interval))
        for bnd in position.spines.keys():
                position.spines[bnd].set_linewidth('1.5')
        position.tick_params(direction = 'out',  length = 3, width = 1.5, labelsize = 10)

        cb.set_label('PSU',fontsize = 12) 
        plt.savefig(nl_Workspaces[iwork].workspace_subdir+
                    '/fig_evolution/2012sss_%s.jpeg'%(str(it)),dpi=300,bbox_inches='tight')
        plt.close()

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
